# Mike's Capstone Project:
## *Analyze Chicago Community Areas using Foursquare and crime data*

### The following code initializes the libraries

In [44]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

from bs4 import BeautifulSoup
import requests

!conda install -c conda-forge geocoder -y
import geocoder

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### *The below code retrieves and recasts data from Wikipedia as a pandas dataframe*
1. The source code of the Wikipedia page for Chicago Community Areas is pulled
2. BeautifulSoup is then used to scrub the data and find in the source code the multiple tables of Community Areas.  All of the rows of the tables are found with the 'tr' tag
3. Next, a list (wikidata) is assembled by looping through the rows in the Wikipedia tables
4. The list is converted into a pandas dataframe with the column headers of Number, Community Area, and Neighborhood
5. Finally, the last 2 characters of each row in the Neighborhood column are removed, deleting the new line tag

In [45]:
source = requests.get('https://en.wikipedia.org/wiki/Community_areas_in_Chicago').text
soup = BeautifulSoup(source, 'lxml')
wikidata = []
a = len(soup.find_all('table', class_='wikitable'))

table = soup.find('table', class_='wikitable')
table_rows = table.find_all('tr')[1:]

for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    wikidata.append(row)

for x in range(0,a-1):
    table = table.find_next('table', class_='wikitable')
    table_rows = table.find_all('tr')[1:]

    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        wikidata.append(row)

wikidata = pd.DataFrame(wikidata, columns=["Number","Community Area","Neighborhood"])
wikidata['Number'] = wikidata['Number'].str[:-1]
wikidata['Community Area'] = wikidata['Community Area'].str[:-1]
wikidata['Neighborhood'] = wikidata['Neighborhood'].str[:-1]
wikidata['Neighborhood'] = wikidata['Neighborhood'].str[1:]
wikidata['Neighborhood'] = wikidata['Neighborhood'].replace("",wikidata['Community Area'])
wikidata['Neighborhood'] = wikidata['Neighborhood'].str.replace("\n", ", ")
wikidata

,Number,Community Area,Neighborhood
0,08,Near North Side,"Cabrini–Green, The Gold Coast, Goose Island, M..."
1,32,Loop,"Loop, Near East Side, South Loop, West Loop Gate"
2,33,Near South Side,"Dearborn Park, Printer's Row, South Loop, Prai..."
3,05,North Center,"Horner Park, Roscoe Village"
4,06,Lake View,"Boystown, Lake View East, Graceland West, Sout..."
5,07,Lincoln Park,"Old Town Triangle, Park West, Ranch Triangle, ..."
6,21,Avondale,"Belmont Gardens, Chicago's Polish Village, Kos..."
7,22,Logan Square,"Belmont Gardens, Bucktown, Kosciuszko Park, Pa..."
8,01,Rogers Park,East Rogers Park
9,02,West Ridge,"Arcadia Terrace, Peterson Park, West Rogers Park"


### *The below code adds location data to the dataframe*
1. The OSM data from geocoder was generally the most accurate, but we noted a number of data points that were off
2. For the data points that were off, the arcgis data was used, replacing the data from the OSM data
  * 4 of the data points from OSM defaulted to streets of the same name versus the Community Area, while the majority of the arcgis data points were farther off
3. The latitude and longitude data were appended to the dataframe

In [46]:
lnglat = wikidata['Community Area'].tolist()
string = ', Chicago'
lnglat = [x + string for x in lnglat]

In [47]:
latitude = []
longitude = []

for ca in lnglat:
    g = geocoder.osm(ca)
    lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    
sodeer = geocoder.arcgis('South Deering, Chicago')
sodeer_coords = sodeer.latlng
sodeer_lat = sodeer_coords[0]
sodeer_lon = sodeer_coords[1]

linsqu = geocoder.arcgis('Lincoln Square, Chicago')
linsqu_coords = linsqu.latlng
linsqu_lat = linsqu_coords[0]
linsqu_lon = linsqu_coords[1]

lakeview = geocoder.arcgis('Lake View, Chicago')
lakeview_coords = lakeview.latlng
lakeview_lat = lakeview_coords[0]
lakeview_lon = lakeview_coords[1]

lincpark = geocoder.arcgis('Lincoln Park, Chicago')
lincpark_coords = lincpark.latlng
lincpark_lat = lincpark_coords[0]
lincpark_lon = lincpark_coords[1]

In [48]:
wikidata["Latitude"] = latitude
wikidata["Longitude"] = longitude
wikidata.loc[66,'Longitude'] = sodeer_lon
wikidata.loc[66,'Latitude'] = sodeer_lat
wikidata.loc[11, 'Longitude'] = linsqu_lon
wikidata.loc[11, 'Latitude'] = linsqu_lat
wikidata.loc[4, 'Longitude'] = lakeview_lon
wikidata.loc[4, 'Latitude'] = lakeview_lat
wikidata.loc[5, 'Longitude'] = lincpark_lon
wikidata.loc[5, 'Latitude'] = lincpark_lat
wikidata

,Number,Community Area,Neighborhood,Latitude,Longitude
0,08,Near North Side,"Cabrini–Green, The Gold Coast, Goose Island, M...",41.900033,-87.634497
1,32,Loop,"Loop, Near East Side, South Loop, West Loop Gate",41.875562,-87.624421
2,33,Near South Side,"Dearborn Park, Printer's Row, South Loop, Prai...",41.856700,-87.624774
3,05,North Center,"Horner Park, Roscoe Village",41.956107,-87.679160
4,06,Lake View,"Boystown, Lake View East, Graceland West, Sout...",41.939820,-87.656820
5,07,Lincoln Park,"Old Town Triangle, Park West, Ranch Triangle, ...",41.921840,-87.647440
6,21,Avondale,"Belmont Gardens, Chicago's Polish Village, Kos...",41.938921,-87.711168
7,22,Logan Square,"Belmont Gardens, Bucktown, Kosciuszko Park, Pa...",41.928400,-87.706764
8,01,Rogers Park,East Rogers Park,42.010531,-87.670748
9,02,West Ridge,"Arcadia Terrace, Peterson Park, West Rogers Park",42.003548,-87.696243


In [49]:
wikidata = wikidata.sort_values(by=['Number']).reset_index(drop = True)
wikidata

,Number,Community Area,Neighborhood,Latitude,Longitude
0,01,Rogers Park,East Rogers Park,42.010531,-87.670748
1,02,West Ridge,"Arcadia Terrace, Peterson Park, West Rogers Park",42.003548,-87.696243
2,03,Uptown,"Buena Park, Argyle Street, Margate Park, Sheri...",41.966630,-87.655546
3,04,Lincoln Square,"Ravenswood, Ravenswood Gardens, Rockwell Crossing",41.975700,-87.689140
4,05,North Center,"Horner Park, Roscoe Village",41.956107,-87.679160
5,06,Lake View,"Boystown, Lake View East, Graceland West, Sout...",41.939820,-87.656820
6,07,Lincoln Park,"Old Town Triangle, Park West, Ranch Triangle, ...",41.921840,-87.647440
7,08,Near North Side,"Cabrini–Green, The Gold Coast, Goose Island, M...",41.900033,-87.634497
8,09,Edison Park,Edison Park,42.005733,-87.814004
9,10,Norwood Park,"Big Oaks, Old Norwood Park, Oriole Park, Union...",41.985590,-87.800575


### *The below code generates a map of Chicago with a dot for each of the community areas*

In [50]:
address = 'Chicago, Illinois'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude1 = location.latitude
longitude1 = location.longitude

# create map using latitude and longitude values
map_chicago = folium.Map(location=[latitude1, longitude1], zoom_start=10)

# add markers to map
for lat, lng, ca, neighborhood in zip(wikidata['Latitude'], wikidata['Longitude'], wikidata['Community Area'], wikidata['Neighborhood']):
    label = '{} | {}'.format(neighborhood, ca)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


#### Define Foursquare Credentials and Version, then show the latitude and longitude of the first Community Area

In [51]:
CLIENT_ID = 'D4PNOLNSZWZWGHSJLZ24T1NTRBRVD3PAKOE3O3E0ULJRYSPA' # your Foursquare ID
CLIENT_SECRET = 'XSJNHP1HHG31QZ0BBTBJV1ERWYQVU03NU2K1JCOE3ODINDUY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D4PNOLNSZWZWGHSJLZ24T1NTRBRVD3PAKOE3O3E0ULJRYSPA
CLIENT_SECRET:XSJNHP1HHG31QZ0BBTBJV1ERWYQVU03NU2K1JCOE3ODINDUY


In [52]:
neighborhood_latitude = wikidata.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = wikidata.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = wikidata.loc[0, 'Community Area'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rogers Park are 42.01053135, -87.6707481966481.


#### This gets the top 100 venues that are with 100 meters of the Rogers Park Community Area.

In [53]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

Here we borrow the **get_category_type** function from the Foursquare lab.

In [54]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Here we clean the json and structure it into a *pandas* dataframe.

In [55]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,El Famous Burrito,Mexican Restaurant,42.010421,-87.674204
1,Morse Fresh Market,Grocery Store,42.008087,-87.667041
2,The Common Cup,Coffee Shop,42.007797,-87.667901
3,Rogers Park Social,Bar,42.007360,-87.666265
4,Lifeline Theatre,Theater,42.007372,-87.666284


#### This function repeats the above process to all of the Community Areas in Chicago

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Community Areas', 
                  'Community Area Latitude', 
                  'Community Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
chicago_venues = getNearbyVenues(names = wikidata['Community Area'],
                                   latitudes = wikidata['Latitude'],
                                   longitudes = wikidata['Longitude']
                                  )

Rogers Park
West Ridge
Uptown
Lincoln Square
North Center
Lake View
Lincoln Park
Near North Side
Edison Park
Norwood Park
Jefferson Park
Forest Glen
North Park
Albany Park
Portage Park
Irving Park
Dunning
Montclare
Belmont Cragin
Hermosa
Avondale
Logan Square
Humboldt Park
West Town
Austin
West Garfield Park
East Garfield Park
Near West Side
North Lawndale
South Lawndale
Lower West Side
Loop
Near South Side
Armour Square
Douglas
Oakland
Fuller Park
Grand Boulevard
Kenwood
Washington Park
Hyde Park
Woodlawn
South Shore
Chatham
Avalon Park
South Chicago
Burnside
Calumet Heights
Roseland
Pullman
South Deering
East Side
West Pullman
Riverdale
Hegewisch
Garfield Ridge
Archer Heights
Brighton Park
McKinley Park
Bridgeport
New City
West Elsdon
Gage Park
Clearing
West Lawn
Chicago Lawn
West Englewood
Englewood
Greater Grand Crossing
Ashburn
Auburn Gresham
Beverly
Washington Heights
Mount Greenwood
Morgan Park
O'Hare
Edgewater


#### This checks the size of the resulting dataframe

In [58]:
print(chicago_venues.shape)
#chicago_venues.head()
#chicago_venues.to_csv("venues.csv")

(3950, 7)


#### Analyzes each neighborhood

In [59]:
# one hot encoding
chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chicago_onehot['Community Areas'] = chicago_venues['Community Areas']

# move neighborhood column to the first column
fixed_columns = [chicago_onehot.columns[-1]] + list(chicago_onehot.columns[:-1])
chicago_onehot = chicago_onehot[fixed_columns]

chicago_onehot.head()

,Community Areas,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Arts Building,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Floating Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Library,Light Rail Station,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Night Market,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rin

#### Here we group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [60]:
chicago_grouped = chicago_onehot.groupby('Community Areas').mean().reset_index()
#chicago_grouped.to_csv("output.csv")
chicago_grouped

,Community Areas,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Arts Building,College Quad,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Floating Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Library,Light Rail Station,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Night Market,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,Road,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rin

#### Here we print each Community Area along with the top 5 most common venues

In [61]:
num_top_venues = 5

for hood in chicago_grouped['Community Areas']:
    print("----"+hood+"----")
    temp = chicago_grouped[chicago_grouped['Community Areas'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albany Park----
                       venue  freq
0         Mexican Restaurant  0.06
1  Middle Eastern Restaurant  0.05
2             Sandwich Place  0.04
3              Grocery Store  0.04
4                       Park  0.04


----Archer Heights----
                  venue  freq
0    Mexican Restaurant  0.09
1     Mobile Phone Shop  0.09
2  Fast Food Restaurant  0.06
3         Grocery Store  0.06
4   Rental Car Location  0.03


----Armour Square----
                venue  freq
0  Chinese Restaurant  0.18
1         Pizza Place  0.05
2                 Bar  0.05
3  Mexican Restaurant  0.04
4         Bus Station  0.04


----Ashburn----
                  venue  freq
0    Seafood Restaurant  0.09
1  Fast Food Restaurant  0.09
2          Intersection  0.09
3                  Park  0.09
4            Donut Shop  0.09


----Auburn Gresham----
                  venue  freq
0  Fast Food Restaurant  0.14
1        Discount Store  0.11
2              Pharmacy  0.07
3                Lounge  0.07


#### This identifies the top 5 most common venues at each neighborhood

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [63]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Community Areas']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Community Areas'] = chicago_grouped['Community Areas']

for ind in np.arange(chicago_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chicago_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Community Areas,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Albany Park,Mexican Restaurant,Middle Eastern Restaurant,Grocery Store,Park,Fast Food Restaurant
1,Archer Heights,Mobile Phone Shop,Mexican Restaurant,Fast Food Restaurant,Grocery Store,Donut Shop
2,Armour Square,Chinese Restaurant,Pizza Place,Bar,Italian Restaurant,Park
3,Ashburn,Park,Fast Food Restaurant,Seafood Restaurant,Donut Shop,Intersection
4,Auburn Gresham,Fast Food Restaurant,Discount Store,Park,Pharmacy,Lounge
5,Austin,Train Station,BBQ Joint,Gymnastics Gym,Fast Food Restaurant,Breakfast Spot
6,Avalon Park,Fast Food Restaurant,Pharmacy,Sandwich Place,Park,Grocery Store
7,Avondale,Bar,Mexican Restaurant,Ice Cream Shop,Park,Donut Shop
8,Belmont Cragin,Mexican Restaurant,Sandwich Place,Pharmacy,Breakfast Spot,Discount Store
9,Beverly,Pizza Place,Cosmetics Shop,Fast Food Restaurant,Discount Store,Women's Store


## Adds historic crime data results to the dataframe and then runs *k*-means to cluster into 5 clusters
1. I separately downloaded the list of all crimes reported in Chicago from 2009 through 2018
2. In a separate notebook, this data was synthesized into the total number crimes per Community Area for the periods 2009-2013 and 2014-Present
3. I then calculated the percentage increase/decrease in reported crimes, and appended that to the dataframe
4. *k*-means was then utilized and a new map was generated to show the Community Areas denoted by their clusters

In [64]:
kclusters = 5
chicago_grouped_clustering = chicago_grouped.drop('Community Areas', 1)
crimedata = pd.read_csv("data.csv")
crimedata = crimedata.drop(['Unnamed: 0', 'Community Area', '2009-2013','2014-2018'], axis=1)
chicago_grouped_clustering1 = chicago_grouped_clustering.join(crimedata)
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(chicago_grouped_clustering1)
kmeans.labels_

array([1, 1, 1, 0, 0, 0, 0, 4, 3, 4, 1, 3, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 4, 3, 4, 4, 0, 1, 4, 4, 1, 4, 4, 1, 1, 1, 1, 4, 1, 3, 4,
       1, 1, 1, 1, 1, 4, 1, 1, 1, 4, 1, 1, 1, 1, 0, 2, 1, 0, 1, 3, 3, 1,
       1, 1, 0, 3, 0, 0, 3, 0, 1, 4, 1], dtype=int32)

In [65]:
chicago_merged = wikidata
chicago_merged['Cluster Labels'] = kmeans.labels_
chicago_merged = chicago_merged.join(neighborhoods_venues_sorted.set_index('Community Areas'), on='Community Area')
chicago_merged

,Number,Community Area,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,01,Rogers Park,East Rogers Park,42.010531,-87.670748,1,Mexican Restaurant,Sandwich Place,Bakery,Pizza Place,Park
1,02,West Ridge,"Arcadia Terrace, Peterson Park, West Rogers Park",42.003548,-87.696243,1,Indian Restaurant,Pakistani Restaurant,Grocery Store,Park,Fast Food Restaurant
2,03,Uptown,"Buena Park, Argyle Street, Margate Park, Sheri...",41.966630,-87.655546,1,Vietnamese Restaurant,Coffee Shop,Thai Restaurant,Sushi Restaurant,Bar
3,04,Lincoln Square,"Ravenswood, Ravenswood Gardens, Rockwell Crossing",41.975700,-87.689140,0,Bar,Café,Sandwich Place,Pizza Place,Korean Restaurant
4,05,North Center,"Horner Park, Roscoe Village",41.956107,-87.679160,0,Bar,Pizza Place,Coffee Shop,Dive Bar,Pub
5,06,Lake View,"Boystown, Lake View East, Graceland West, Sout...",41.939820,-87.656820,0,Sandwich Place,Japanese Restaurant,Spa,Pizza Place,Bar
6,07,Lincoln Park,"Old Town Triangle, Park West, Ranch Triangle, ...",41.921840,-87.647440,0,Italian Restaurant,Pizza Place,Coffee Shop,Sushi Restaurant,American Restaurant
7,08,Near North Side,"Cabrini–Green, The Gold Coast, Goose Island, M...",41.900033,-87.634497,4,American Restaurant,Hotel,Gym,Steakhouse,Coffee Shop
8,09,Edison Park,Edison Park,42.005733,-87.814004,3,Pizza Place,Bar,Italian Restaurant,Park,Nail Salon
9,10,Norwood Park,"Big Oaks, Old Norwood Park, Oriole Park, Union...",41.985590,-87.800575,4,Pizza Place,Park,American Restaurant,Hobby Shop,Bar


In [66]:
address = 'Chicago, Illinois'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude1 = location.latitude
longitude1 = location.longitude

# create map
map_clusters = folium.Map(location=[latitude1, longitude1], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chicago_merged['Latitude'], chicago_merged['Longitude'], chicago_merged['Community Area'], chicago_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


# The above map shows interesting similarities between the neighborhoods
1. I first ran this without the crime data, which showed a very different clustering that looked much more random
2. Adding the crime data shows more likeness among Community Areas downtown and along the Lake, and the difference between areas further away from downtown
3. An interesting point to note is the Chatham and the Pullman Community Areas belong to a similar cluster as closer to downtown.  Upon further investigation, these two Community Areas have experienced noteable changes and that are on rail lines directly to downtown